# Run Spec2Vec Training Flow Locally

This notebook shows how Spec2Vec training flow can be run locally. Output is trained model path.

## Imports

In [1]:
from omigami.spectra_matching.spec2vec.factory import Spec2VecFlowFactory
import mlflow

## Build Training Flow

You can adjust following parameters of your choice:
- `flow-name`: Flow name
- `iterations`: Spec2Vec model parameter
- `window`: Spec2Vec model parameter
- `intensity_weighting_power`: Spec2Vec model parameter
- `allowed_missing_percentage`: Spec2Vec model parameter
- `n_decimals`: Spec2Vec model parameter
- `dataset_id`: dataset to train Spec2Vec model on, will download the data from a particular URL. Available options are:
  - `small`: data from https://raw.githubusercontent.com/MLOps-architecture/share/main/test_data/SMALL_GNPS.json. This data is not up-to-date with GNPS.
  - `small_500`: data from https://raw.githubusercontent.com/MLOps-architecture/share/main/test_data/SMALL_GNPS_500_spectra.json. This data is not up-to-date with GNPS.
  - `10k`: This dataset has no url, but it uses first 10k spectra from GNPS. This data is not up-to-date with GNPS.
  - `complete`: data from https://gnps-external.ucsd.edu/gnpslibrary/ALL_GNPS.json. This will always be up-to-datw with GNPS.
- `ion_mode`: `"positive"` or `"negative"`

Rest of the parameters can be stay as it is, as they are related to tools used in developing the flow.

In [2]:
from pathlib import Path
storage_root = Path.cwd() / "results"

In [3]:
factory = Spec2VecFlowFactory(
    storage_root=storage_root,
    model_registry_uri="sqlite:///mlflow.sqlite",
    mlflow_output_directory=str(storage_root / "spec2vec/models"),
)
flow = factory.build_training_flow(
    flow_name="spec2vec",
    iterations=5,
    window=10,
    intensity_weighting_power=0.5,
    allowed_missing_percentage=5.0,
    n_decimals=2,
    dataset_id="small",
    ion_mode="positive",
    project_name="spec2vec-positive",
)
flow

/Users/czanella/miniconda-r/envs/omigami/lib/python3.7/contextlib.py:119: UserWarning: Tasks were created but not added to the flow: {<Task: Task>}. This can occur when `Task` classes, including `Parameters`, are instantiated inside a `with flow:` block but not added to the flow either explicitly or as the input to another task. For more information, see https://docs.prefect.io/core/advanced_tutorials/task-guide.html#adding-tasks-to-flows.
  next(self.gen)


<Flow: name="spec2vec">

## Run Training Flow

In [4]:
flow_run = flow.run()

[2022-01-13 12:12:31-0300] INFO - prefect.FlowRunner | Beginning Flow run for 'spec2vec'
[2022-01-13 12:12:31-0300] INFO - prefect.TaskRunner | Task 'DownloadData': Starting task run...
[2022-01-13 12:12:32-0300] INFO - prefect.DownloadData | Downloaded 100 spectra from https://raw.githubusercontent.com/MLOps-architecture/share/main/test_data/SMALL_GNPS.json to /Users/czanella/dev/datarevenue/omigami-core/notebooks/training/results/datasets/small/gnps.json.
[2022-01-13 12:12:32-0300] INFO - prefect.DownloadData | Saving spectrum ids to /Users/czanella/dev/datarevenue/omigami-core/notebooks/training/results/datasets/small/spectrum_ids.pkl
[2022-01-13 12:12:32-0300] INFO - prefect.TaskRunner | Task 'DownloadData': Finished task run for task with final state: 'Success'
[2022-01-13 12:12:32-0300] INFO - prefect.TaskRunner | Task 'CreateChunks': Starting task run...
[2022-01-13 12:12:32-0300] INFO - prefect.CreateChunks | Loading file /Users/czanella/dev/datarevenue/omigami-core/notebooks/t

Successfully registered model 'spec2vec-model'.
2022/01/13 12:12:36 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: spec2vec-model, version 1


[2022-01-13 12:12:36-0300] INFO - prefect.RegisterModel | Created model run_id: f523919f81134dd88f30caf99fb1bfe3.
[2022-01-13 12:12:36-0300] INFO - prefect.TaskRunner | Task 'RegisterModel': Finished task run for task with final state: 'Success'
[2022-01-13 12:12:36-0300] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


Created version '1' of model 'spec2vec-model'.


## Output Model Path

In [5]:
register_task = flow.get_tasks("RegisterModel")[0]
run_id = flow_run.result[register_task].result

artifact_uri = mlflow.get_run(run_id).info.artifact_uri
model_uri = f"{artifact_uri}/model/python_model.pkl"

In [6]:
print(f"Spec2Vec model is available at: {model_uri}")

Spec2Vec model is available at: /Users/czanella/dev/datarevenue/omigami-core/notebooks/training/results/spec2vec/models/f523919f81134dd88f30caf99fb1bfe3/artifacts/model/python_model.pkl
